---

# <font size="6"> Data Conversion and Extraction File</font>  
<font size="3"> *By: Tristan Yuen*   
*Date: December 15th, 2020* </font>

In [ ]:
#import starting libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [ ]:
#Opening json file from my computer
with open ("C:\\Users\\Arkateks\\Desktop\\BrainStation Files\\Capstone\\spotify_million_playlist_dataset\\data\\mpd.slice.2000-2999.json") as json_file:
    data = json.load(json_file)

<font size="5"> Data Exploration

In [ ]:
#Let's see what the data shows
data

From here we can see the entire dataset is nested within a dictionary. Within the overall dictionary, there seems to be a two dictioanries, one that includes just general information about this json file while the other dictionary holds all the playlists within a list. Furthermore, the list in itself containts a dictionary who's keys are various factors about the playlist itself. We can see in the first one in terms of features being: name of playlist, if it is collaborative or not, playlist id, last changed, number of songs, number of different albums, how many followers, and tracks. Now tracks initself is a list of a nested dictionary of all the different songs within the playlist and its own brief metadata. The data includes the position of the track within the playlist, artist name, the track id for spotify, album id for spotify, the name of the song, as well as how long is it and the album name. Each of these tracks is its own dictionary. We can see that there are many layers of data within here like an onion.

In [ ]:
#initial test to see how to parse through the data
print(data["playlists"][2]["tracks"][0])

Here is an attempt to iterate through this list to see how I would extract the artist names from the list. We can see here I've managed to reach the playlist level within the dataset. However, the key thing to note is what the two integers represent here. The first integer shows which playlist is it in terms of of position. So in the case above it would be the third playlist of this entire set. The second integer represents the position of the track and here it would the first track of the playlist. 

In [ ]:
data["playlists"][0]["tracks"][0]["artist_name"]

Following through with what I did above, I see that I need to use the `artist_name` key to access the artist name within the dataset.

Artists names in a list

In [ ]:
#creating a for loop to gather all artist name into a list
#setting up an empty list variable
artist = []

#for loop that goes 3 levels deep from our test above to extract the data needed
for playlist in data["playlists"]:
    for track in playlist["tracks"]:
        artist.append(track["artist_name"])

In [ ]:
#Check to see if for loop ran correctly
artist

Here we can see the entire list of all the artists that are within this json file.

In [ ]:
#let's see how many totol artists are there
u_artists = list(set(artist))
len(u_artists)

There are 9861 unique artists within this playlist dataset. Below in the commentted out code was the first iteration of the code I used to find the number of unique artists within the list and I shortened it with the above.

In [ ]:
#unique_artists = []

#for names in artist: 
    # check if exists in unique_list or not 
    #if names not in unique_artists: 
        #unique_artists.append(names) 
        
#len(unique_artists)

There is not much data cleaning to do as this data was given as part of a challenge from spotify, so there is comfort in the fact that they purposed the data to focus more on the results, so we know that the data is clean and there would be no need to search for missing data of any sorts

Now we know at this point that we want to use the DictVectorizer is in tandem with the Counter function within collections. Counter allows us to count how many of each artist which is in a string format. Afterwards DictVectorizer takes the strings along with the count that the Counter creates. The artist will become the key and the value will be the count of how many times the artist appears within the playlist.

In [ ]:
#Creating the for loop to extract the artists names

#empty dictionary name to store all the playlists with all the artists names
dict = {}

#empty list to store the artists names
artists_names =[]

#iterating through a range of 1000 as each json file has 1000 playlists
for i in range(1000):
    dict[i] = artists_names 
    j = len(data["playlists"][i]["tracks"])    #this helps us make sure that the first dictionary ends when the first playlist ends so that when we need to iterate through a new playlist it creates a new dictionary
    artists_names = []
    for n in range(j):
        artists_names.append(data["playlists"][i]["tracks"][n]["artist_name"])

In [ ]:
#See how long the dictionary is in its first position
len(dict[1])

In [ ]:
#number of tracks in playlist 1
data["playlists"][0]["num_tracks"]

We can see from the above that our for loop ran successfully through a quick check as the length of the playlist match with the data given within the dataset.

In [ ]:
#import Counter from the collections library
from collections import Counter

#setting up empty list to store our values
artists_cnt = []

#running a for loop to count how many times an artist appears within a playlist
for i in range(1000):
    artists_title = Counter(dict[i])
    artists_cnt.append(artists_title)
    
artists_cnt

From aboe we can see now that each position within the list is its own playlist. Within each position is the counter variable that has all the counts embedded within a dictionary where the keys are the artists and the values are how many times the artist appears within the playlist.

In [ ]:
#removing the first position from the list as it is an empty Counter variable
del artists_cnt[0]

In [ ]:
#importing DictVectorizer to help us change the above to a sparse matrix 
from sklearn.feature_extraction import DictVectorizer

In [ ]:
#Setting our DictVecotrizer as a variable
v = DictVectorizer()

In [ ]:
#Fitting our list into Dictvectorizer
v.fit(artists_cnt)

In [ ]:
#transforming the data once the it has been fitted and setting it as a variable
surprise_trial = v.transform(artists_cnt)
surprise_trial

In [ ]:
#Here setting a variable for plotting later on of all the feature names within the sparse matrix which will be all the artists within the list that we created before
names = v.get_feature_names()

In [ ]:
# We can extract the information and put it in a data frame to make it easier to see what has occured
df = pd.DataFrame(columns=v.get_feature_names(), data=artists_cnt)
display(df)

We see how the sparse matrix is made out now within a dataframe. Interestingly we see some unique artists names as DictVectorizer seems to have sorted all the artists into alphabetical order starting with all the symbols first. 

Now we see if the dataframe is accurate.

In [ ]:
#isolate for a playlist which is a row within the dataframe
playlist_997 = df.loc[997, :]

In [ ]:
# setting up a quick equation to see if the count of the playlist 997 matches the one within the list we created before
Counter(playlist_997[playlist_997.notna()].to_dict()) == artists_cnt[997]

The above equation ran true so we move on to show visuals about the data.

In [ ]:
# Setting all the NaN's in the dataframe above to 0 for numpy calculations
df_zeroes = df.replace(np.nan,0)

If I want to see which is the top artists from the above sparse matrix, I can't do that as the NaN's stop me from using any numpy calculations. As a result, we have to convert them in order to do some quick visualizations for our data.

Top 10 Artists chart

In [ ]:
#setting a new row on the bottom to tally all the times an artists appears
artists_total  = pd.DataFrame(df_zeroes.sum(axis = 0), columns = ['Total'])
artists_total.reset_index(level = 0 , inplace = True)
artists_total = artists_total.sort_values(by = 'Total', ascending = False)

Here we created a new row on the bottom of the dataframe to calculate how many times an artist appeared within this dataset. We had to do the row as all the columns are the artists leading for the calculation to be done in a vertical sense.

In [ ]:
#plot top 10 artists in playlist
plt.figure(figsize = (15,5))
plt.bar(artists_total['index'].head(10), artists_total['Total'].head(10), color = 'forestgreen')
plt.xlabel('Artists')
plt.ylabel('Count')
plt.xticks(rotation = 45)
plt.title('Top 10 Artists in Playlist Set 3')
plt.show()

With the above chart, we can see that the top 10 artists within this playlist is mainly hip hop and pop artists. Not surprisingly here, because the Spotify dataset is 1 million playlists that were curated between 2010 and 2017 and all the playlists originate from North America. 

Statistics

In [ ]:
#empty list to store values
playlist_len = []

#for loop to extract the length of each playlist and put its numeric value into a list
for i in range(999):
    length_playlist = len(artists_cnt[i])
    playlist_len.append(length_playlist)

In [ ]:
avg = sum(playlist_len)/len(playlist_len)
print("The average is ", round(avg,2))

The average length of a playlist seems to be about ~38 songs. This seems about right as numerous articles I have read in terms of creating the prefect playlist say that the range should be around 30-50 songs. 

In [ ]:
#empty list to store values
songs = []

#for loop for extracitng the number of songs within this playlist dataset
for playlist in data["playlists"]:
    for track in playlist["tracks"]:
        songs.append(track["track_name"])

In [ ]:
#finding how many unique songs are there
u_song = list(set(songs))
len(u_song)

There is 30,230 unique songs within this playlist. This is a good amount of diversity within this dataset and its only 1 of 1000 json files.

<font size="5"> Song Vectorizer section

Below is the exact same code above. However, this time it instead of drawing out the artists from the dataset and putting them into a sparse matrix with how many times they appear, it is with song titles intead. Ultimately, I did not use any of th section below for my model, but it would be interesting to come back and see how the model would perform with a much larger matrix and a much more specific recommendation. As the bottom section is not used I will not be commenting what each part of the code does.

---

In [ ]:
dict_songs = {}

songs_names =[]

for i in range(1000):
    dict_songs[i] = songs_names
    j = len(data["playlists"][i]["tracks"])
    songs_names = []
    for n in range(j):
        songs_names.append(data["playlists"][i]["tracks"][n]["track_name"])

In [ ]:
songs_cnt = []

for i in range(1000):
    songs_title = Counter(dict_songs[i])
    songs_cnt.append(songs_title)

In [ ]:
del songs_cnt[0]

In [ ]:
vs = DictVectorizer()

In [ ]:
vs.fit(songs_cnt)

In [ ]:
vs.transform(songs_cnt)

In [ ]:
titles = vs.get_feature_names()

In [ ]:
# We can extract the information and put it in a data frame to make it easier to see what has occured
df_songs = pd.DataFrame(columns=vs.get_feature_names(), data=songs_cnt)
display(df_songs)# We can extract the information and put it in a data frame to make it easier to see what has occured


check

In [ ]:
playlist_997_s = df_songs.loc[997, :]
Counter(playlist_997_s[playlist_997_s.notna()].to_dict()) == songs_cnt[997]

In [ ]:
df_zeroes_songs = df_songs.replace(np.nan,0)

In [ ]:
df_zeroes_songs 

Song charts

In [ ]:
songs_total  = pd.DataFrame(df_zeroes_songs.sum(axis = 0), columns = ['Total'])
songs_total.reset_index(level = 0 , inplace = True)
songs_total = songs_total.sort_values(by = 'Total', ascending = False)

In [ ]:
songs_total

In [ ]:
plt.figure(figsize = (15,5))
plt.bar(songs_total['index'].head(10), songs_total['Total'].head(10), color = 'cornflowerblue')
plt.xlabel('Song Title')
plt.ylabel('Count')
plt.xticks(rotation = 45)
plt.title('Top 10 Songs in Playlist Set 3')
plt.show()

song statistics

In [ ]:
song_duration = []

for playlist in data["playlists"]:
    for track in playlist["tracks"]:
        song_duration.append(track["duration_ms"])

In [ ]:
avg = (sum(song_duration)/len(song_duration))/1000/60
print("The average is song duration is  ", round(avg,2))

---

<font size="5"> First iteration of the model

In [ ]:
#importing necessary libraries from surprise
from surprise import Dataset
from surprise.reader import Reader
from surprise.prediction_algorithms.matrix_factorization import SVD as FunkSVD

In [ ]:
#creating empty list to store values
playlist_num = []
artist = []
rating = []

#iterating through each key within the dictionary and inserting the proper value into its own list
for i, counter in enumerate(artists_cnt):
    for key, value in counter.items():
        playlist_num.append(i)
        artist.append(key)
        rating.append(value)

In [ ]:
#taking lists above and creating its own dictionary and then putting it into a dataframe
a_dict = {"Playlist":playlist_num, "Artist":artist, "Rating":rating}
df_trial = pd.DataFrame(a_dict)
df_trial

Within the surprise library for a recommendation system it requires a dataframe to be 3 columns. Either the first or second column has to be the user id while the other is the item id. The third column would be the given rating for that user id combination. As a result, I had to condense my sparse matrix dataframe into 3 lists for values and took those lists and created new keys for each of the lists and created a new dataframe for it.

In [ ]:
#Checks the max rating that will be inputted
df_trial['Rating'].max()

In [ ]:
#checks the minimum rating that will be inputted
df_trial['Rating'].min()

In [ ]:
#using the functions built into surprise to load in my dataframe
my_dataset = Dataset.load_from_df(df_trial, Reader(rating_scale=(1, 135)))
my_train_dataset = my_dataset.build_full_trainset()

In [ ]:
#Running a quick model of SVD to see if the model will run
my_algorithm = FunkSVD(n_factors=10, 
                       n_epochs=100, 
                       lr_all=0.00001,    # Learning rate for each epoch
                       biased=False,  # This forces the algorithm to store all latent information in the matrices
                       verbose=1)

my_algorithm.fit(my_train_dataset)

In [ ]:
#importing new libaries to evaluate the model that was run
from surprise import accuracy
from surprise.model_selection import train_test_split

# The surprise package doesn't allow you to test on the trainset we built
my_train_dataset, my_test_dataset = train_test_split(my_dataset, test_size=0.5)

#setting a predictions variable and running a test to see the results
predictions = my_algorithm.test(my_test_dataset)

In [ ]:
#seeing what the predictions look like
predictions

In [ ]:
# RMSE
RMSE = accuracy.rmse(predictions, verbose=False)
print(RMSE)

In [ ]:
# MAE
MAE = accuracy.mae(predictions, verbose=False)
print(MAE)

In [ ]:
FCP = accuracy.fcp(predictions, verbose=0)
print(FCP) # ideal is 1.0, worst is 0.0

<font size="5"> Initial Grid Search

In [ ]:
#import grid search function from surprise
from surprise.model_selection import GridSearchCV

#establish parameters grid
param_grid = {'n_epochs': [50, 100, 150, 200, 250], 
              'lr_all': [0.001, 0.0001, 0.000001],
              'reg_all': [0.1, 0.2, 0.3, 0.4]}

#run grid search with cross validation of 5 folds
gs_trial = GridSearchCV(FunkSVD, param_grid, measures=['rmse', 'mae'], cv=5)

In [ ]:
#fitting the grid search with my dataframe
%%time
gs_trial.fit(my_dataset)

In [ ]:
# combination of parameters that gave the best RMSE score
print(gs_trial.best_params['rmse'])

In [ ]:
# best RMSE score
print(gs_trial.best_score['rmse'])

Here we can a gridsearch and received the best parameters. However, other than the numer of epochs which stood at 150 which was in the middle of my range, the learning rate best parameter was on the edge at 0.001 and the regularization at 0.3. As a result, we don't know for sure if those are the best parameters as both cases seem to be edge cases. As a result, to get the best parameters we will have to do a secondary grid search where we fine tune it even more by putting in more specific parameters.

In [ ]:
#check dataframe for reference
df_trial

In [ ]:
#finding the minimum value of the entire rating system
df_trial['Rating'].min(axis = 0)

I ran the above code as well with .max() function. and the result was 135. The minimum rating for the dataset is 1 and the max rating is 135.

In [ ]:
#testing out an initial equation of min max scaler
((135 - 1) / (df_trial['Rating'].max(axis = 0 ) - 0)) * 9 + 1

In [ ]:
#create the min max scaler function within a for loop so that it can iterate through my dataframe
%%time 

min = 1
max = 10 
num_rows = len(df_trial['Rating'])
scaled_scores = []

#iterating through the dataframe
for i in range(1000):
    if df_trial['Playlist'][i] == [i]:
        for i in range(num_rows):
            std_playlist = (df_trial['Rating'][i]- min) / (df_trial['Rating'].max(axis = 0) - min) #taking the min max scaler function mathematical equation and inputting into the for loop for iteration
            scaled_rating = round(std_playlist * (max - min) + min) #second part of the min max scaler function 
            scaled_scores.append(scaled_rating)
    

In [ ]:
#printing new min and max ratings
df_trial['Scaled_Ratings'] = scaled_scores
print(df_trial['Scaled_Ratings'].max())
print(df_trial['Scaled_Ratings'].min())

In [ ]:
#dropping the unnecessary rating column as surprise library only takes in 3 columns 
df_scaled = df_trial.drop(['Rating'], axis = 1)
df_scaled

Here in the above section, I realized that surprise may not fun not function as well due to the fact that my rating system is extremely broad. The minimum value is 1 and the maximum value is 135. As a result, surprise may have difficulty finding an accurate function. As a result, I created a for loop so that the ratings would scale based on each playlist. The min max scaler function would be applied on a playlist level which is why I could not just use the library function as it would have scaled it based on the entire dataframe instead of scaling per playlist. 

In [ ]:
#importing all the algorithms within the surprise library 
from surprise.prediction_algorithms.matrix_factorization import SVD as FunkSVD
from surprise.prediction_algorithms.random_pred import NormalPredictor 
from surprise.prediction_algorithms.baseline_only import BaselineOnly
from surprise.prediction_algorithms.knns import KNNBasic
from surprise.prediction_algorithms.knns import KNNWithMeans
from surprise.prediction_algorithms.knns import KNNWithZScore
from surprise.prediction_algorithms.knns import KNNBaseline
from surprise.prediction_algorithms.matrix_factorization import NMF
from surprise.prediction_algorithms.slope_one import SlopeOne
from surprise.prediction_algorithms.co_clustering import CoClustering
from surprise.prediction_algorithms.matrix_factorization import SVDpp
from surprise.model_selection import cross_validate

In [ ]:
#loading new Dataframe
data = Dataset.load_from_df(df_scaled, Reader(rating_scale=(1, 10)))

In [ ]:
%%time

benchmark = []
# Iterate over all algorithms
for algorithm in [FunkSVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['rmse'], cv=5, verbose=True)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)

#make dataframe for all my tests scores
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')    

I ran all of the models and tested them based on residual mean scores to see which model would be the best. From the above dataframe you can see that Baseline Only was the best performing one in terms of residual mean scores while Normal Predictor was the worst. Interestingly, between BaselineOnly and SVD there is not a significant difference between the residual mean scores. The difference is really only 0.1. As a result, when I was deciding on which model to use I landed with SVD. There are two factors. The first factor being that for SVD I am able to explore the latent variables that come with running an SVD variable. The second reason is that SVD within surprise utilizes stochastic gradient descent. The benefits of using SGD is that it is known to be able to converge comparatively more than other methodologies. As a result, when I scale the data up using AWS, using this model will hopefully provide better results. Also comparatively, within the test times, I chose SVD over KNN as computationally it is much less intensive.

In [ ]:
#reoptimize gridsearch after initial grid search
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [60, 70, 80, 90, 100, 110, 120], 
              'lr_all': [0.1, 0.01, 0.001, 0.0001],
              'reg_all': [0, 0.025, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4]}
gs_trial = GridSearchCV(FunkSVD, param_grid, measures=['rmse', 'mae'], cv=5)

In [ ]:
%%time
gs_trial.fit(data)

In [ ]:
# combination of parameters that gave the best RMSE score
print(gs_trial.best_params['rmse'])

This is the second time we run the grid search and this time we see better results wih the best paramter not being near the edge. Moving forward we will continue to use these paramaeters for our FunkSVD algorithm.

In [ ]:
#reference dataframe once again
df_scaled

In [ ]:
#seeing approximately how many times Drake appears within a playlist
print(df_scaled.loc[df_scaled['Artist'] == 'Drake'])

In [ ]:
#checked a random Playlist to see the ratings overall
print(df_scaled.loc[df_scaled['Playlist'] == 512])

In [ ]:
#loading new Dataframe
data = Dataset.load_from_df(df_scaled, Reader(rating_scale=(1, 10)))
train_data = data.build_full_trainset()

In [ ]:
#running model after the hyperparamater optimization
my_algorithm = FunkSVD(n_factors=10, 
                       n_epochs=110, 
                       lr_all=0.001, # Learning rate for each epoch
                       reg_all = 0.25,
                       biased=False,  # This forces the algorithm to store all latent information in the matrices
                       verbose=1)

my_algorithm.fit(train_data)

In [ ]:
#using the predict function to see the estimated scores 
my_algorithm.predict(uid = 50, iid = 'Rihanna')

Afte running the model with the scaled ratings as well as the tuned hyperparameters we use the predict function to see what would a user within the given training dataset rate an artist of our choice. As a result, I ran through multiple iterations and found that most of the estimates were rated 1. As a result, I found this extremely peculiar and checked with the above the ratings and found that most of the ratings within all of the playlist were 1. This tells me that most people only puts an artist within their playlist only once. It seems to me that I will need to change up how the rating system works or else all my top 10 predictions will be random in a sense since everything is rated 1.

In [ ]:
#creating variable to house the user latent variables
U = my_algorithm.pu
U.shape

In [ ]:
#creating variable to house the item latent variables 
M = my_algorithm.qi.T
M.shape

In [ ]:
#isolating for the first user profile 
inner_user_id = train_data.to_inner_uid(1) # find the inner representation of user 1
user_profile = U[inner_user_id]
user_profile

In [ ]:
#putting the latent of a user variables in a plot
plt.figure(figsize=(15, 7))
plt.barh([f'Latent Var{i}' for i in range(1,len(first_song)+1)], user_profile)
plt.title("Latent Variable Profile of User #1")
plt.ylabel("Latent Variable")
plt.xlabel("Value")
plt.show()

In [ ]:
#putting the latent of item variables in a plot
inner_songs_id = train_data.to_inner_iid('Drake') # find the inner representation of item 1
song_profile = M[:, inner_songs_id]
song_profile

In [ ]:
#plotting out the item variables
plt.figure(figsize=(15, 7))
plt.barh([f'Latent Var{i}' for i in range(1,len(first_song)+1)], song_profile)
plt.title("Latent Variable Profile of User #1")
plt.ylabel("Latent Variable")
plt.xlabel("Value")
plt.show()

In [ ]:
#multiplying the user and item arrays to see the expected ratings
expected_rating = np.dot(user_profile, song_profile)
expected_rating

I decided to check the latent variables to see if anything stood out. However, due to the nature of my dataframe, it would be extremely hard to check the latent variables visually. Possibly as next steps, I could create a function to parse the necessary information so I could look deeper into it. However, for this prototype we will not be investigating it.

In [ ]:
#plotting out distribution of scaled ratings
plt.figure()
plt.bar(df_scaled['Scaled_Ratings'].value_counts().index,df_scaled['Scaled_Ratings'].value_counts())
plt.show()

In [ ]:
#scaled ratings distribution in numeric form
df_scaled['Scaled_Ratings'].value_counts()

Looking further into this scaling problem, I plotted out the distribution of the scaling and not to mmy surpise 99% of the ratings is 1 out of 10. Found out the exact numbers numerically, and it is not a suprise (pun intended) that my prediction function was predicting an estimate of 1 consistently considering 99% of the artists were rated 1 out of 10 by the users. 

In [ ]:
#distribution f ratings prior to standard scaler
plt.figure(figsize = (30,10))
plt.title(' Distribution of Ratings within the Playlist', fontsize = 25)
plt.xlabel('Ratings', fontsize = 20)
plt.ylabel('Count of Ratings', fontsize = 20)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.bar(df_trial['Rating'].value_counts().index,df_trial['Rating'].value_counts(), color = 'cornflowerblue')
plt.show()

Looking deeper into the scaling issue, the standard scaler formula keeps the variance the same no matter the range. As a result, if post-scaling my distribution is skewed, theoretically pre-scaling my distribution should be just as skewed. As a result, I plotted it to see my distribution before the scaling and not surprisngly we can see that the distribution is extremely skewed towardsthe rating of 1. Consequently, I will have to fix this issue for my final model. 